# Running cellpose 2.0 in colab with a GPU

<font size = 4>Cellpose 2.0 now allows you to train your own models in the GUI!

This notebook allows you to load this **custom model** and run the model on your images with a GPU. 

In this notebook, you can also **train** a custom model using your labels (`_seg.npy`) files, or other labels as `_masks.tif` files. If you already have a trained model, skip this part of the notebook.

For more details on cellpose 2.0 check out the [paper](https://www.biorxiv.org/content/10.1101/2022.04.01.486764v1) or the [talk](https://www.youtube.com/watch?v=3ydtAhfq6H0).

Mount your google drive to access all your image files, segmentations, and custom models. This also ensures that any models you train are saved to your google drive. If you'd like to try out the notebook without your own files, please download the sample images from tissuenet (optional step in Setup below).

This notebook was inspired by the Zero-Cost Deep-Learning to Enhance Microscopy project (https://github.com/HenriquesLab/DeepLearning_Collab/wiki). Jointly developed by the Jacquemet (link to https://cellmig.org/) and Henriques (https://henriqueslab.github.io/) laboratories. Please check out their great work!

# Setup

We will first install cellpose 2.0, check the GPU is working, and mount google drive to get your models and images.

## Installation

In [ ]:
# !pip install "opencv-python-headless<4.3"
# !pip install cellpose

Check CUDA version and that GPU is working in cellpose and import other libraries.

In [1]:
!nvcc --version
!nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Thu_Jun_11_22:26:48_Pacific_Daylight_Time_2020
Cuda compilation tools, release 11.0, V11.0.194
Build cuda_11.0_bu.relgpu_drvr445TC445_37.28540450_0
Thu Aug 25 14:47:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.48       Driver Version: 451.48       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 207... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   63C    P8     9W /  N/A |    286MiB /  8192MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
          

C:\Users\nicol\Anaconda3\envs\cellpose\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> GPU activated? YES


## Evaluate on test data (optional)

If you have test data, check performance

In [ ]:
# # get files (during training, test_data is transformed so we will load it again)
# output = io.load_train_test_data(test_dir, mask_filter='_seg.npy')
# test_data, test_labels = output[:2]

# # run model on test images
# masks = model.eval(test_data, 
#                    channels=[chan, chan2],
#                    diameter=diam_labels)[0]

# # check performance using ground truth labels
# ap = metrics.average_precision(test_labels, masks)[0]
# print('')
# print(f'>>> average precision at iou threshold 0.5 = {ap[:,0].mean():.3f}')


plot masks

In [ ]:

# plt.figure(figsize=(12,8), dpi=150)
# for k,im in enumerate(test_data):
#     img = im.copy()
#     plt.subplot(3,len(train_files), k+1)
#     img = np.vstack((img, np.zeros_like(img)[:1]))
#     img = img.transpose(1,2,0)
#     plt.imshow(img)
#     plt.axis('off')
#     if k==0:
#         plt.title('image')

#     plt.subplot(3,len(train_files), len(train_files) + k+1)
#     plt.imshow(masks[k])
#     plt.axis('off')
#     if k==0:
#         plt.title('predicted labels')

#     plt.subplot(3,len(train_files), 2*len(train_files) + k+1)
#     plt.imshow(test_labels[k])
#     plt.axis('off')
#     if k==0:
#         plt.title('true labels')
# plt.tight_layout()

# Use custom model to segment images

Take custom trained model from above, or upload your own model to google drive / colab runtime.

## Parameters

In [3]:
# model name and path

#@markdown ###Custom model path (full path):

model_path = "Z:\\people\\gritti\\projects\\Jordi\\code\\test_cellpose3d\\individual_planes\\models\\CP_Jordi" #@param {type:"string"}

#@markdown ###Path to images:

dir = "Z:\\people\\gritti\\projects\\Jordi\\code\\test_cellpose3d\\2022-06-02_155713" #@param {type:"string"}

#@markdown ###Channel Parameters:

Channel_to_use_for_segmentation = "Grayscale" #@param ["Grayscale", "Blue", "Green", "Red"]

# @markdown If you have a secondary channel that can be used, for instance nuclei, choose it here:

Second_segmentation_channel= "None" #@param ["None", "Blue", "Green", "Red"]


# Here we match the channel to number
if Channel_to_use_for_segmentation == "Grayscale":
  chan = 0
elif Channel_to_use_for_segmentation == "Blue":
  chan = 3
elif Channel_to_use_for_segmentation == "Green":
  chan = 2
elif Channel_to_use_for_segmentation == "Red":
  chan = 1


if Second_segmentation_channel == "Blue":
  chan2 = 3
elif Second_segmentation_channel == "Green":
  chan2 = 2
elif Second_segmentation_channel == "Red":
  chan2 = 1
elif Second_segmentation_channel == "None":
  chan2 = 0

#@markdown ### Segmentation parameters:

#@markdown diameter of cells (set to zero to use diameter from training set):
diameter =  0#@param {type:"number"}
#@markdown threshold on flow error to accept a mask (set higher to get more cells, e.g. in range from (0.1, 3.0), OR set to 0.0 to turn off so no cells discarded):
flow_threshold = 0.4 #@param {type:"slider", min:0.0, max:3.0, step:0.1}
#@markdown threshold on cellprob output to seed cell masks (set lower to include more pixels or higher to include fewer, e.g. in range from (-6, 6)):
cellprob_threshold=0 #@param {type:"slider", min:-6, max:6, step:1}


## run custom model

how to run the custom model in a notebook

In [10]:
# gets image files in dir (ignoring image files ending in _masks)
files = io.get_image_files(dir, '_masks')
print(files)
images = [io.imread(f) for f in files]

# declare model
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)

# use model diameter if user diameter is 0
diameter = model.diam_labels if diameter==0 else diameter

# run model on test images
masks, flows, styles = model.eval(
                                  images, 
                                  channels=[chan, chan2],
                                  diameter=diameter,
                                  flow_threshold=flow_threshold,
                                  cellprob_threshold=cellprob_threshold,
                                  do_3D=True
                                  )

['Z:\\people\\gritti\\projects\\Jordi\\code\\test_cellpose3d\\2022-06-02_155713\\C1-stream-1-1.tif']


100%|███████████████████████████████████████████████████████████████████████████████| 276/276 [00:01<00:00, 215.56it/s]


## save output to *_seg.npy

you will see the files save in the Files tab and you can download them from there

In [11]:
from cellpose import io

io.masks_flows_to_seg(images, 
                      masks, 
                      flows, 
                      diameter*np.ones(len(masks)), 
                      files, 
                      channels=[chan, chan2])

## save output masks to tiffs/pngs or txt files for imageJ

In [12]:
io.save_masks(images, 
              masks, 
              flows, 
              files, 
              channels=[chan, chan2],
              png=False, # save masks as PNGs and save example image
              tif=True, # save masks as TIFFs
              save_txt=True, # save txt outlines for ImageJ
              save_flows=False, # save flows as TIFFs
              save_outlines=False, # save outlines as TIFFs 
              )
    